# 🌌 Mini-Cosmos: Training in Colab

This notebook trains the VAE and World Model on Google Colab GPU.

**Steps:**
1. Connect to Google Drive
2. Clone repository
3. Upload/download data
4. Train model
5. Save checkpoints to Drive

## 1. Setup

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create project folder in Drive
!mkdir -p /content/drive/MyDrive/mini-cosmos
!mkdir -p /content/drive/MyDrive/mini-cosmos/checkpoints
!mkdir -p /content/drive/MyDrive/mini-cosmos/data

In [ ]:
# Clone repository
%cd /content
!git clone https://github.com/Danya-rrr/mini-cosmos.git
%cd mini-cosmos

In [ ]:
# Install dependencies
!pip install torch torchvision --quiet
!pip install pytorch-lightning einops omegaconf tqdm matplotlib --quiet

## 2. Upload Data

**Option A:** Upload ZIP file to Google Drive, then unzip here

**Option B:** Upload directly to Colab (slower, resets on disconnect)

In [ ]:
# Option A: Copy data from Google Drive
# First, upload your data.zip to Google Drive /mini-cosmos/data/

# Uncomment and run:
# !cp /content/drive/MyDrive/mini-cosmos/data/data.zip /content/mini-cosmos/
# !unzip -q data.zip
# !rm data.zip

In [ ]:
# Option B: Upload directly (for small datasets)
# from google.colab import files
# uploaded = files.upload()  # Select your data.zip

In [ ]:
# Check data structure
!ls -la data/
!ls data/raw/ | head -10
!ls data/processed/nuscenes/ | head -10

## 3. Test Dataset

In [ ]:
# Test that dataset loads correctly
!python src/data/dataset.py

## 4. Train VAE

In [ ]:
# Train VAE
!python scripts/train_vae.py --epochs 50 --batch_size 16 --lr 1e-4

In [ ]:
# Copy checkpoints to Google Drive (so they persist)
!cp -r outputs/checkpoints/* /content/drive/MyDrive/mini-cosmos/checkpoints/

## 5. Visualize Results

In [ ]:
# Visualize VAE reconstruction
!python scripts/visualize_vae.py

In [ ]:
# Show saved images
from IPython.display import Image, display
import os

viz_dir = 'outputs/visualizations'
if os.path.exists(viz_dir):
    for img_file in os.listdir(viz_dir):
        if img_file.endswith('.png'):
            print(f"\n{img_file}:")
            display(Image(filename=os.path.join(viz_dir, img_file)))

In [ ]:
# Copy visualizations to Drive
!cp -r outputs/visualizations/* /content/drive/MyDrive/mini-cosmos/checkpoints/

## 6. Training History

In [ ]:
import json
import matplotlib.pyplot as plt

# Load history
with open('outputs/vae_history.json', 'r') as f:
    history = json.load(f)

# Plot losses
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Total Loss')
axes[0].legend()

axes[1].plot(history['train_recon'], label='Train')
axes[1].plot(history['val_recon'], label='Val')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Reconstruction Loss')
axes[1].legend()

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/mini-cosmos/checkpoints/training_history.png')
plt.show()

## 7. Download Results

In [ ]:
# Zip and download checkpoints
!zip -r checkpoints.zip outputs/checkpoints/

from google.colab import files
files.download('checkpoints.zip')